In [1]:
require 'nn'
require 'optim'

torch.manualSeed(0)
torch.setnumthreads(4)

In [2]:
function setup() 
    classes = { 1, 2, 3, 4, 5, 6, 7, 8, 9, 10 }
    geometry = { 32, 32 }

    net = nn.Sequential()

    net:add(nn.SpatialConvolution(1, 6, 5, 5))
    net:add(nn.ReLU())
    net:add(nn.SpatialMaxPooling(2, 2, 2, 2))

    net:add(nn.SpatialConvolution(6, 16, 5, 5))
    net:add(nn.ReLU())
    net:add(nn.SpatialMaxPooling(2, 2, 2, 2))

    net:add(nn.View(16*5*5))
    net:add(nn.Linear(16*5*5, 120))
    net:add(nn.ReLU())
    net:add(nn.Linear(120, 84))
    net:add(nn.ReLU())
    net:add(nn.Linear(84, #classes))
    net:add(nn.LogSoftMax())
    
    parameters, gradParameters = net:getParameters()
    criterion = nn.ClassNLLCriterion()
    confusion = optim.ConfusionMatrix(classes)
end

In [3]:
function normalize(input, n_channels)
    local mean = {}
    local stdev = {}
    
    for channel = 1, n_channels do
        mean[channel] = input.data[{ {}, {channel}, {}, {} }]:mean()
        stdev[channel] = input.data[{ {}, {channel}, {}, {} }]:std()
        
        print('Channel ' .. channel .. ' mean: ' .. mean[channel] .. ' stdev: ' .. stdev[channel])
        
        input.data[{ {}, {channel}, {}, {} }]:add(-mean[channel])
        input.data[{ {}, {channel}, {}, {} }]:div(stdev[channel])
    end
end

In [4]:
function load_data()    
    train = torch.load('mnist-p1b-train.t7')
    test = torch.load('mnist-p1b-test.t7')
    
    n_train = train.data:size()[1]
    n_test = test.data:size()[1]
    
    train.data = train.data:double()
    test.data = test.data:double()
end

In [5]:
function exec_training(obj, n_channels, current_epoch) 
    confusion:zero()
    
    for t = 1, n_train, batch_size do
        local limit = math.min(t + batch_size - 1, n_train)
        
        local limited_batch_size = limit - t + 1
        local inputs = torch.Tensor(limited_batch_size, n_channels, geometry[1], geometry[2])
        local targets = torch.Tensor(limited_batch_size)
        local k = 1
        
        for i = t, limit do
            local input = obj.data[i]:clone()
            local target = obj.label[i]
            
            inputs[k] = input
            targets[k] = target
            k = k + 1
        end
        
        local feval = function(x)
            collectgarbage()
            
            if x ~= parameters then
                parameters:copy(x)
            end
            gradParameters:zero()
            
            local outputs = net:forward(inputs)
            local f = criterion:forward(outputs, targets)
            local df_do = criterion:backward(outputs, targets)
            net:backward(inputs, df_do)
            
            for i = 1, limited_batch_size do
                confusion:add(outputs[i], targets[i])
            end
            
            return f, gradParameters
        end
        
        sgd_state = sgd_state or {
            learningRate = 0.03,
            learningRateDecay = 1e-7
        }
        optim.sgd(feval, parameters, sgd_state)
    end
    
    confusion:updateValids()
    return confusion.averageValid
end

In [6]:
function exec_testing(obj, n_channels, current_epoch)
    confusion:zero()
    
    for t = 1, n_test, batch_size do
        local limit = math.min(t + batch_size - 1, n_test)
        
        local limited_batch_size = limit - t + 1
        local inputs = torch.Tensor(limited_batch_size, n_channels, geometry[1], geometry[2])
        local targets = torch.Tensor(limited_batch_size)
        local k = 1
        
        for i = t, limit do
            local input = obj.data[i]:clone()
            local target = obj.label[i]
            
            inputs[k] = input
            targets[k] = target
            k = k + 1
        end
        
        local preds = net:forward(inputs)
        
        for i = 1, limited_batch_size do
            confusion:add(preds[i], targets[i])
        end
    end 
    
    confusion:updateValids()
    return confusion.averageValid
end

In [7]:
setup()
load_data()
normalize(train, 1)
normalize(test, 1)

n_epoch = 32
batch_size = 256
local last_train_conf
local last_test_conf
for epoch = 1, n_epoch do
    local acc_train = exec_training(train, 1, epoch)
    
    if (epoch == n_epoch) then
        last_train_conf = confusion:__tostring__()
    end
    
    local acc_test = exec_testing(test, 1, epoch)
    
    if (epoch == n_epoch) then
        last_test_conf = confusion:__tostring__()
    end
    
    io.write(string.format('Epoch %3d: %.4f%% | %.4f%%\n', epoch, acc_train * 100, acc_test * 100))
end

print(last_train_conf)
print(last_test_conf)

Channel 1 mean: 25.509416422526 stdev: 70.180423838273	


Channel 1 mean: 25.87140625 stdev: 70.739298703524	


Epoch   1: 60.8162% | 86.9976%


Epoch   2: 92.2206% | 93.0819%


Epoch   3: 95.3835% | 95.5764%


...

Epoch  28: 99.4882% | 98.5494%


Epoch  29: 99.5149% | 98.5782%


Epoch  30: 99.5434% | 98.5310%


Epoch  31: 99.5673% | 98.5723%


Epoch  32: 99.6026% | 98.5315%
ConfusionMatrix:
[[    5913       1       2       1       1       0       2       0       1       2]   99.831% 	[class: 1]
 [       0    6723       1       0       1       0       3      10       4       0]   99.718% 	[class: 2]
 [       3       3    5943       0       1       1       0       6       1       0]   99.748% 	[class: 3]
 [       0       0       3    6102       0       7       0       2      13       4]   99.527% 	[class: 4]
 [       0       2       0       0    5824       0       4       2       1       9]   99.692% 	[class: 5]
 [       1       1       1       5       1    5393      11       0       2       6]   99.483% 	[class: 6]
 [       3       5       0       0       1       5    5900       0       4       0]   99.696% 	[class: 7]
 [       0       9       7       0       2       0       0    6240       1       6]   99.601% 	[class: 8]
 [       1       4       4       6       2       3       4       3    5816       8]   99.402% 	[class: 9